# Day 02: Hello World with YAML

**用 YAML 配置文件构建你的第一个 AI Agent，无需编写代码！**

## 课程来源
- [Advent of Agents 2025 - Day 2](https://adventofagents.com/)
- [ADK Config Agents Documentation](https://google.github.io/adk-docs/agents/config/)
- [Third Party MCP Tools in ADK](https://google.github.io/adk-docs/tools/third-party/)

## 学习目标

1. 理解 ADK Agent Config（YAML 配置）的概念
2. 使用 YAML 创建简单的 AI Agent
3. 了解 YAML Agent 的配置选项
4. 运行和测试 YAML Agent
5. 学习如何添加工具（Python 和 MCP）

---
## 1. 什么是 ADK Agent Config？

ADK Agent Config 是 Google ADK 的一个**实验性功能**，让你可以用 YAML 文件定义 AI Agent，而不需要编写 Python 代码。

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     传统 Python 方式 vs YAML 方式                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Python 方式 (Day 1):                                                  │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │  from google.adk.agents import Agent                        │       │
│   │                                                              │       │
│   │  agent = Agent(                                              │       │
│   │      model="gemini-2.0-flash",                              │       │
│   │      name="assistant",                                       │       │
│   │      instruction="You are a helpful assistant."             │       │
│   │  )                                                           │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                                                          │
│   YAML 方式 (Day 2):                                                    │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │  # root_agent.yaml                                          │       │
│   │  name: assistant                                            │       │
│   │  model: gemini-2.0-flash                                    │       │
│   │  instruction: You are a helpful assistant.                   │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                                                          │
│   两种方式都能创建相同功能的 Agent！                                      │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### YAML 方式的优势

| 优势 | 说明 |
|------|------|
| **无需编码** | 非程序员也能创建 Agent |
| **快速原型** | 几行配置即可运行 |
| **易于修改** | 直接编辑 YAML 文件 |
| **版本控制友好** | 纯文本格式，易于 diff |

---
## 2. 环境设置

In [1]:
# 安装依赖
# !uv pip install google-adk python-dotenv

import sys
import os
from pathlib import Path

# 添加项目根目录到路径
sys.path.insert(0, str(Path.cwd().parent))

# 加载 API Key
from shared import get_api_key
os.environ["GOOGLE_API_KEY"] = get_api_key()

print("✅ 环境配置完成！")
print(f"   Python 版本: {sys.version.split()[0]}")

✅ 环境配置完成！
   Python 版本: 3.14.2


In [2]:
# 验证 google-adk 安装
import google.adk as adk

print(f"✅ google-adk 已安装")
print(f"   版本: {adk.__version__ if hasattr(adk, '__version__') else '未知'}")

✅ google-adk 已安装
   版本: 1.21.0


---
## 3. YAML Agent 基本结构

一个最简单的 YAML Agent 只需要几行配置：

In [3]:
# 查看我们的 YAML Agent 配置
yaml_content = Path("root_agent.yaml").read_text()
print("📄 root_agent.yaml 内容：")
print("=" * 50)
print(yaml_content)

📄 root_agent.yaml 内容：
# yaml-language-server: $schema=https://raw.githubusercontent.com/google/adk-python/refs/heads/main/src/google/adk/agents/config_schemas/AgentConfig.json
name: search_agent
model: gemini-2.5-flash
description: A helpful assistant for user questions.
instruction: Answer user questions to the best of your knowledge



### YAML 配置字段说明

| 字段 | 必需 | 说明 |
|------|------|------|
| `name` | ✅ | Agent 的标识符名称 |
| `model` | ✅ | 使用的 Gemini 模型 |
| `description` | ❌ | Agent 的简短描述 |
| `instruction` | ❌ | 系统提示词/行为指令 |
| `tools` | ❌ | Agent 可使用的工具列表 |

---
## 4. 使用 Python 加载 YAML Agent

虽然 YAML Agent 主要通过 `adk web` 运行，但我们也可以在 Python 中加载它：

In [4]:
# 方法 1: 使用传统 Python API 实现相同功能
from google import genai

client = genai.Client()

# 这等同于我们 YAML 中定义的 Agent
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="你好！请用中文介绍一下你自己。"
)

print("🤖 Agent 响应：")
print(response.text)

🤖 Agent 响应：
你好！很高兴认识你。

我是一个大型语言模型，由 Google 训练。你可以把我当成一个智能助手或者聊天机器人。我的主要功能是理解和生成人类语言，并利用这些能力来回答你的问题、创作文本、翻译语言、总结信息等等。

我没有实体，也没有个人身份。我存在于计算机服务器中，通过程序代码来实现各种功能。

你可以问我任何问题，我会尽我所能地回答。不过，请记住，我仍在不断学习和改进中，有时候可能会犯错。请不要把我的回答当成绝对真理，也请不要利用我来做任何非法的事情。

总而言之，我是一个乐于助人、知识渊博的语言模型，期待和你一起探索语言的奥秘！ 你想让我做什么呢？



In [5]:
# 方法 2: 使用 ADK Agent 类（Python 方式）
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai.types import Content, Part
import asyncio

# 创建与 YAML 等效的 Agent
agent = Agent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="You are a helpful assistant that can answer user questions. Always provide clear and concise responses in the user's language."
)

print(f"✅ Agent 创建成功")
print(f"   名称: {agent.name}")
print(f"   模型: {agent.model}")

✅ Agent 创建成功
   名称: search_agent
   模型: gemini-2.0-flash


In [6]:
# 运行 Agent
async def run_agent():
    runner = InMemoryRunner(
        agent=agent,
        app_name="day_02_demo"
    )

    # 创建会话
    session = await runner.session_service.create_session(
        app_name="day_02_demo",
        user_id="user_1"
    )

    # 发送消息
    user_message = Content(
        role="user",
        parts=[Part(text="什么是机器学习？请用一句话解释。")]
    )

    print("👤 用户: 什么是机器学习？请用一句话解释。")
    print("")

    # 获取响应
    async for event in runner.run_async(
        user_id="user_1",
        session_id=session.id,
        new_message=user_message
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"🤖 Agent: {part.text}")

# 运行
await run_agent()

👤 用户: 什么是机器学习？请用一句话解释。

🤖 Agent: 机器学习是一种让计算机在没有明确编程的情况下，从数据中学习并做出预测或决策的方法。



---
## 5. 运行 YAML Agent（命令行方式）

YAML Agent 最常用的运行方式是通过 `adk web` 命令：

```bash
# 在项目根目录运行
cd /path/to/25-Day-Agents-Course-by-Google
uv run adk web
```

然后在浏览器中：
1. 打开 http://127.0.0.1:8000
2. 从下拉菜单选择 `day_02`
3. 开始对话！

In [7]:
# 显示目录结构
print("📁 Day 02 目录结构：")
print()
print("day_02/")
print("├── __init__.py          # 标记为 Python 包（ADK 需要）")
print("├── root_agent.yaml      # YAML Agent 配置")
print("├── hello_world_yaml.ipynb  # 本 Notebook")
print("└── README.md            # 学习文档")
print()
print("💡 注意：文件夹名必须使用下划线 (_)，不能用连字符 (-)")

📁 Day 02 目录结构：

day_02/
├── __init__.py          # 标记为 Python 包（ADK 需要）
├── root_agent.yaml      # YAML Agent 配置
├── hello_world_yaml.ipynb  # 本 Notebook
└── README.md            # 学习文档

💡 注意：文件夹名必须使用下划线 (_)，不能用连字符 (-)


---
## 6. 高级配置：添加工具

### 6.1 添加 Python 工具

如果不使用 `tools.py` 文件，可以将工具直接写在 `__init.py` 中，使用的时候直接使用 [文件夹名.工具名]

可以创建 `tools.py` 文件，然后在 YAML 中引用 使用的时候直接使用 [文件夹名.tools.工具名]：

In [12]:
# 示例：创建自定义工具
tools_code = '''# tools.py

def get_current_time() -> str:
    """获取当前时间。"""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression: str) -> str:
    """计算数学表达式。

    Args:
        expression: 数学表达式，如 "2 + 2"

    Returns:
        计算结果
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"计算错误: {e}"
'''

print("📄 tools.py 示例：")
print("=" * 50)
print(tools_code)

📄 tools.py 示例：
# tools.py

def get_current_time() -> str:
    """获取当前时间。"""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression: str) -> str:
    """计算数学表达式。

    Args:
        expression: 数学表达式，如 "2 + 2"

    Returns:
        计算结果
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"计算错误: {e}"



In [13]:
# 对应的 YAML 配置
yaml_with_tools = '''# root_agent.yaml - 带工具版本

name: assistant_with_tools
model: gemini-2.0-flash
description: A helpful assistant with custom tools.
instruction: |
  You are a helpful assistant.
  Use get_current_time to tell the current time.
  Use calculate for math operations.

tools:
  - name: tools.get_current_time
  - name: tools.calculate
'''

print("📄 带工具的 YAML 配置：")
print("=" * 50)
print(yaml_with_tools)

📄 带工具的 YAML 配置：
# root_agent.yaml - 带工具版本

name: assistant_with_tools
model: gemini-2.0-flash
description: A helpful assistant with custom tools.
instruction: |
  You are a helpful assistant.
  Use get_current_time to tell the current time.
  Use calculate for math operations.

tools:
  - tools.get_current_time
  - tools.calculate



### 6.2 添加 MCP 工具

MCP (Model Context Protocol) 允许连接外部工具服务器：

In [10]:
# MCP 工具配置示例
yaml_with_mcp = '''# root_agent.yaml - 带 MCP 工具版本

name: agent_with_mcp
model: gemini-2.0-flash
description: An agent with MCP tools.
instruction: |
  You are a helpful assistant.
  You can check the current time using the time tool.
tools:
  - name: MCPToolset
    args:
      stdio_server_params:
        command: "uvx"
        args:
        - "mcp-server-time"

'''

print("📄 带 MCP 工具的 YAML 配置：")
print("=" * 50)
print(yaml_with_mcp)
print()
print("💡 使用 MCP 前需要安装对应的服务器：")
print("   pip install mcp-server-time")

📄 带 MCP 工具的 YAML 配置：
# root_agent.yaml - 带 MCP 工具版本

name: agent_with_mcp
model: gemini-2.0-flash
description: An agent with MCP tools.
instruction: |
  You are a helpful assistant.
  You can check the current time using the time tool.

tools:
  - type: MCPToolset
    stdio_server_params:
      command: uvx
      args:
        - mcp-server-time


💡 使用 MCP 前需要安装对应的服务器：
   pip install mcp-server-time


---
## 7. 多 Agent 配置

YAML 也支持定义多个协作的 Agent：

In [11]:
# 多 Agent 配置示例
multi_agent_yaml = '''# root_agent.yaml - 多 Agent 版本

name: coordinator
model: gemini-2.0-flash
description: Coordinates between researcher and writer.
instruction: |
  You are a coordinator agent.
  Use the researcher for finding information.
  Use the writer for creating content.

sub_agents:
  - name: researcher
    model: gemini-2.0-flash
    description: Researches information.
    instruction: |
      You are a research specialist.
      Find accurate information for queries.

  - name: writer
    model: gemini-2.0-flash
    description: Writes content.
    instruction: |
      You are a content writer.
      Create clear and engaging content.
'''

print("📄 多 Agent YAML 配置：")
print("=" * 50)
print(multi_agent_yaml)

📄 多 Agent YAML 配置：
# root_agent.yaml - 多 Agent 版本

name: coordinator
model: gemini-2.0-flash
description: Coordinates between researcher and writer.
instruction: |
  You are a coordinator agent.
  Use the researcher for finding information.
  Use the writer for creating content.

sub_agents:
  - name: researcher
    model: gemini-2.0-flash
    description: Researches information.
    instruction: |
      You are a research specialist.
      Find accurate information for queries.

  - name: writer
    model: gemini-2.0-flash
    description: Writes content.
    instruction: |
      You are a content writer.
      Create clear and engaging content.



---
## 8. 对比：Python vs YAML

| 特性 | Python 方式 | YAML 方式 |
|------|-------------|----------|
| 学习曲线 | 需要编程知识 | 只需了解 YAML 语法 |
| 灵活性 | 高（完全可编程） | 中等（配置限制） |
| 调试 | 可用 Python 调试器 | 依赖日志和 UI |
| 复杂逻辑 | ✅ 完全支持 | ❌ 有限支持 |
| 快速原型 | 需要编码 | ✅ 几分钟完成 |
| 模型支持 | 所有模型 | 仅 Gemini |
| 状态 | 稳定 | 实验性功能 |

---
## 9. 常见问题与解决

### 问题 1: "Invalid app name"
```
Value error, Invalid app name 'day-02': must be a valid identifier
```
**解决**: 文件夹名使用下划线，不用连字符：`day_02` ✅  `day-02` ❌

### 问题 2: Agent 不在下拉菜单
**解决**: 确保文件夹中有 `__init__.py` 文件

### 问题 3: 工具配置错误
```
Input should be a valid dictionary or instance of ToolConfig
```
**解决**: 检查工具配置格式，内置工具可能需要特定语法

### 问题 4: 命令找不到
```
zsh: command not found: adk
```
**解决**: 使用 `uv run adk web` 或先激活虚拟环境

---
## 10. 核心要点

### 今日学习

| 概念 | 说明 |
|------|------|
| **ADK Agent Config** | 用 YAML 定义 Agent 的实验性功能 |
| **root_agent.yaml** | Agent 配置文件名（固定名称） |
| **基本字段** | name, model, description, instruction |
| **运行方式** | `uv run adk web` 启动 Web UI |
| **命名规则** | 文件夹使用下划线，需要 `__init__.py` |

### 最简配置

```yaml
# root_agent.yaml
name: my_agent
model: gemini-2.0-flash
instruction: You are a helpful assistant.
```

### 运行命令

```bash
# 项目根目录
uv run adk web

# 浏览器打开
http://127.0.0.1:8000
```

---
## 11. 资源链接

### 官方文档
- [ADK Config Agents Documentation](https://google.github.io/adk-docs/agents/config/)
- [Third Party MCP Tools in ADK](https://google.github.io/adk-docs/tools/third-party/)
- [ADK Samples Repository](https://github.com/google/adk-samples)

### 教程
- [2-Minute ADK: Build Agents the Easy Way — YAML](https://medium.com/google-cloud/2-minute-adk-build-agents-the-easy-way-yaml-a55678d64a75)
- [Tutorial: AI Agent with Google Search (YAML)](https://x.com/Saboo_Shubham_/status/1971038699329908885)
- [Tutorial: Multi-agent app with MCP (YAML)](https://x.com/Saboo_Shubham_/status/1971763476818547010)